In [31]:
import pandas as pd
import requests
import selenium
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 
from bs4 import BeautifulSoup
from time import sleep 
import re
import sys 
import os
sys.path.append(os.path.abspath("../src"))   
from datetime import datetime
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors


import funciones as f

Sacar lista supers

In [77]:
url_super = "https://super.facua.org/"
res_facua = requests.get(url_super)
sopa_mercados = BeautifulSoup(res_facua.content, "html.parser")
lista_mercados = sopa_mercados.findAll("a", {"class" :"nav-link"})
lista_supers= [mercado.getText().lower() for mercado in lista_mercados]
lista_supers.pop()
lista_supers


['mercadona', 'alcampo', 'carrefour', 'hipercor', 'dia', 'eroski']

Sacar lista productos

In [78]:
url_mercados = f"https://super.facua.org/mercadona/"
res_producto = requests.get(url_mercados)
sopa_productos = BeautifulSoup(res_producto.content, "html.parser")
lista_prod = sopa_productos.findAll("p", {"class" :"fw-bolder"})
lista_productos= [mercado.getText().lower().replace(" ", "-") for mercado in lista_prod]
for i, producto in enumerate(lista_productos,start=1):
    id_prod= i
    nombre_producto= producto
    print(nombre_producto)
    print(id_prod)


aceite-de-girasol
1
aceite-de-oliva
2
leche
3


Sacar lista variantes


In [79]:
url_variante = "https://super.facua.org/mercadona/aceite-de-girasol/"
res_variante= requests.get(url_variante)
sopa_variante = BeautifulSoup(res_variante.content, "html.parser")
lista_var = sopa_variante.findAll("p", {"class" :"fw-bolder"})
lista_variantes= [variante.getText().lower().replace(" ", "-").replace(",", "").replace("º","").replace(".","") for variante in lista_var]
for i, variante in enumerate(lista_variantes, start=1):
    id_var= i
    nombre_variante= variante
    print(id_var)
    print(nombre_variante)

1
aceite-de-girasol-refinado-02-hacendado-1-l
2
aceite-de-girasol-refinado-02-hacendado-5-l


Lo comento para que no me moleste

In [2]:
# for mercado in lista_supers:
#     df_tablas= pd.DataFrame()
#     driver = webdriver.Chrome()
#     url_mercados = f"https://super.facua.org/{mercado}/"
#     driver.get(url_mercados)
#     sleep(3)
#     #aceptar cookie
#     driver.find_element("css selector", "#rcc-confirm-button").click()
#     #hacer aquí la lista de productos
#     res_producto = requests.get(url_mercados)
#     sopa_productos = BeautifulSoup(res_producto.content, "html.parser")
#     lista_prod = sopa_productos.findAll("p", {"class" :"fw-bolder"})
#     lista_productos= [prod.getText().lower().replace(" ", "-") for prod in lista_prod]

#     for i, producto in enumerate(lista_productos):
#         driver.execute_script("window.scrollBy(0, 200);")
#         sleep(2)
#         #selecciona producto
#         driver.find_element("css selector", f"body > section:nth-child(4) > div > div.row.gx-4.gx-lg-5.row-cols-2.row-cols-md-3.row-cols-xl-4.justify-content-center > div:nth-child({i+1}) > div > div.card-footer.p-4.pt-0.border-top-0.bg-transparent > div > a").click()
#         sopa_variante = BeautifulSoup(driver.page_source, "html.parser")
#         lista_var = sopa_variante.findAll("p", {"class" :"fw-bolder"})
#         lista_variantes= [variante.getText().lower().replace(" ", "-").replace(",", "").replace("º","").replace(".","") for variante in lista_var]
#         lista_variantes_bien=[]
#         for elemento in lista_variantes:
#             if elemento.startswith("aceite") or elemento.startswith("leche"):
#                 lista_variantes_bien.append(elemento)
#         for j ,variante in enumerate(lista_variantes_bien):
#             while True:
#                 #selecciona variante
#                 try:
#                     sleep(2)
#                     driver.find_element("css selector", f"body > section:nth-child(4) > div > div:nth-child(7) > div:nth-child({j+1}) > div > div.card-footer.p-4.pt-0.border-top-0.bg-transparent > div > a").click()
#                     break
#                 except:
#                     driver.execute_script("window.scrollBy(0, 200);")
#             sopa_mercados = BeautifulSoup(driver.page_source, "html.parser")
#             tabla_general= sopa_mercados.findAll("table")
#             tablaindiv=tabla_general[-1]
#             #por aquí crear las columnas necesarias usando lista de productos y variante (mercado se pone formateado tal cual)
#             df_tablas=pd.concat([df_tablas, f.sacar_tabla(tablaindiv,mercado,producto,variante)])
#             driver.back()
#         driver.back()    
#     driver.close()
#     df_tablas.to_csv(f'../datos/datos{mercado}.csv', index=False)


In [3]:
# df_tablas.head()

Carga de datos

In [2]:
alcampo=pd.read_csv("../datos/datosalcampo.csv")
mercadona=pd.read_csv("../datos/datosmercadona.csv")
carrefour=pd.read_csv("../datos/datoscarrefour.csv")
dia=pd.read_csv("../datos/datosdia.csv")
eroski=pd.read_csv("../datos/datoseroski.csv")
hipercor=pd.read_csv("../datos/datoshipercor.csv")

In [3]:
alcampo.head()

,Día,Precio (€),Variación,supermercado,producto,variante
0,02/07/2024,"57,13",=,alcampo,aceite-de-oliva,aceite-de-oliva-virgen-extra-procedente-aceitu...
1,03/07/2024,"57,13",=,alcampo,aceite-de-oliva,aceite-de-oliva-virgen-extra-procedente-aceitu...
2,04/07/2024,"57,13",=,alcampo,aceite-de-oliva,aceite-de-oliva-virgen-extra-procedente-aceitu...
3,05/07/2024,"57,13",=,alcampo,aceite-de-oliva,aceite-de-oliva-virgen-extra-procedente-aceitu...
4,06/07/2024,"57,13",=,alcampo,aceite-de-oliva,aceite-de-oliva-virgen-extra-procedente-aceitu...


In [4]:
mercadona.head()

,Día,Precio (€),Variación,supermercado,producto,variante
0,12/07/2024,"1,45",=,mercadona,aceite-de-girasol,aceite-de-girasol-refinado-02-hacendado-1-l
1,13/07/2024,"1,45",=,mercadona,aceite-de-girasol,aceite-de-girasol-refinado-02-hacendado-1-l
2,14/07/2024,"1,45",=,mercadona,aceite-de-girasol,aceite-de-girasol-refinado-02-hacendado-1-l
3,15/07/2024,"1,45",=,mercadona,aceite-de-girasol,aceite-de-girasol-refinado-02-hacendado-1-l
4,16/07/2024,"1,45",=,mercadona,aceite-de-girasol,aceite-de-girasol-refinado-02-hacendado-1-l


In [5]:
eroski.head()

,Día,Precio (€),Variación,supermercado,producto,variante
0,12/07/2024,"1,89",=,eroski,aceite-de-girasol,aceite-alto-oleico-eroski-botella-1-litro
1,13/07/2024,"1,89",=,eroski,aceite-de-girasol,aceite-alto-oleico-eroski-botella-1-litro
2,14/07/2024,"1,89",=,eroski,aceite-de-girasol,aceite-alto-oleico-eroski-botella-1-litro
3,15/07/2024,"1,89",=,eroski,aceite-de-girasol,aceite-alto-oleico-eroski-botella-1-litro
4,16/07/2024,"1,89",=,eroski,aceite-de-girasol,aceite-alto-oleico-eroski-botella-1-litro


In [ ]:
dia.head()

,Día,Precio (€),Variación,supermercado,producto,variante
0,29/08/2024,"1,99",=,dia,aceite-de-girasol,aceite-de-girasol-alto-oleico-especial-freidor...
1,30/08/2024,"1,99",=,dia,aceite-de-girasol,aceite-de-girasol-alto-oleico-especial-freidor...
2,31/08/2024,"1,99",=,dia,aceite-de-girasol,aceite-de-girasol-alto-oleico-especial-freidor...
3,01/09/2024,"1,99",=,dia,aceite-de-girasol,aceite-de-girasol-alto-oleico-especial-freidor...
4,02/09/2024,"1,99",=,dia,aceite-de-girasol,aceite-de-girasol-alto-oleico-especial-freidor...


In [7]:
hipercor.head()

,Día,Precio (€),Variación,supermercado,producto,variante
0,02/07/2024,"14,94",=,hipercor,aceite-de-oliva,aceite-de-oliva-virgen-extra-picual-casa-junca...
1,11/07/2024,"14,94",=,hipercor,aceite-de-oliva,aceite-de-oliva-virgen-extra-picual-casa-junca...
2,12/07/2024,"14,94",=,hipercor,aceite-de-oliva,aceite-de-oliva-virgen-extra-picual-casa-junca...
3,13/07/2024,"14,94",=,hipercor,aceite-de-oliva,aceite-de-oliva-virgen-extra-picual-casa-junca...
4,14/07/2024,"14,94",=,hipercor,aceite-de-oliva,aceite-de-oliva-virgen-extra-picual-casa-junca...


In [8]:
carrefour.head()

,Día,Precio (€),Variación,supermercado,producto,variante
0,12/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l
1,13/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l
2,14/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l
3,15/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l
4,16/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l


Creación Data Frame Total y Exploración inicial

In [9]:
df= pd.DataFrame()
df_total= pd.concat([df,carrefour,mercadona,eroski,dia,hipercor,alcampo])

In [10]:
df_total.head()

,Día,Precio (€),Variación,supermercado,producto,variante
0,12/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l
1,13/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l
2,14/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l
3,15/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l
4,16/07/2024,"21,40",=,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l


**RESULTADOS INICIALES DE EXPLORACION**    


Contamos con un Data Frame de 59382 filas y 6 columnas.

*Nulos:*  

- No cuenta con ningun dato nulo que tengamos que tratar


*Duplicados:*  

- Contamos con 193 filas duplicadas lo que supone un 0,33% del total de filas.  
- Tras analizarlos y comprobar la web, nos dimos cuenta de que los podemos eliminar ya que en la propia web aparecen productos duplicados(Nos quedamos con un Data Frame con 59189 filas y 6 columnas). 

*Formatos:*  

- En la columna variacion quise convertir los iguales a 0, quedarme solo con los porcentajes y luego ya pasar a float.  
- En la columna precio cambié las , por los . y los pasé a float.   
- En la columna Día la fecha debería estar en formato fecha así que la cambié. 





Tratamos los duplicados

In [11]:
df_total.shape

(59382, 6)

In [12]:
round((df_total.duplicated().sum()/len(df_total))*100, 2)

np.float64(0.33)

In [13]:
df_total[df_total.duplicated()].head()

,Día,Precio (€),Variación,supermercado,producto,variante
11616,15/07/2024,"1,08",=,carrefour,leche,leche-desnatada-central-lechera-asturiana-brik...
11617,16/07/2024,"1,08",=,carrefour,leche,leche-desnatada-central-lechera-asturiana-brik...
11618,17/07/2024,"1,08",=,carrefour,leche,leche-desnatada-central-lechera-asturiana-brik...
11619,18/07/2024,"1,08",=,carrefour,leche,leche-desnatada-central-lechera-asturiana-brik...
11620,19/07/2024,"1,08",=,carrefour,leche,leche-desnatada-central-lechera-asturiana-brik...


In [14]:
df_total.drop_duplicates(inplace=True)

Formateamos

In [15]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59189 entries, 0 to 214
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Día           59189 non-null  object
 1   Precio (€)    59189 non-null  object
 2   Variación     59189 non-null  object
 3   supermercado  59189 non-null  object
 4   producto      59189 non-null  object
 5   variante      59189 non-null  object
dtypes: object(6)
memory usage: 3.2+ MB


In [16]:
df_total["Precio (€)"]=df_total["Precio (€)"].str.replace(",", ".").astype(float)



In [17]:
df_total.sample(5)

,Día,Precio (€),Variación,supermercado,producto,variante
20670,16/10/2024,1.16,=,eroski,leche,leche-semidesnatada-pascual-brik-1-litro
11208,13/08/2024,1.36,=,carrefour,leche,leche-desnatada-carrefour-botella-15-l\n
127,30/07/2024,0.95,=,alcampo,leche,leche-entera-fresca-hacendado-1-l
11922,04/09/2024,1.19,=,carrefour,leche,leche-desnatada-con-vitaminas-a+d-puleva-brik-...
29,27/09/2024,1.99,=,dia,aceite-de-girasol,aceite-de-girasol-alto-oleico-especial-freidor...


In [18]:
df_total["Variación"]=df_total["Variación"].str.replace("+","").str.replace(",", ".").str.split(" ").str[0].str.replace("=", "0").astype(float)

In [19]:
df_total["Día"]=pd.to_datetime(df_total["Día"], dayfirst= True)

In [20]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59189 entries, 0 to 214
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Día           59189 non-null  datetime64[ns]
 1   Precio (€)    59189 non-null  float64       
 2   Variación     59189 non-null  float64       
 3   supermercado  59189 non-null  object        
 4   producto      59189 non-null  object        
 5   variante      59189 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 3.2+ MB


In [21]:
df_total.describe().T

,count,mean,min,25%,50%,75%,max,std
Día,59189,2024-08-29 18:50:14.708814080,2024-06-22 00:00:00,2024-08-03 00:00:00,2024-08-28 00:00:00,2024-09-26 00:00:00,2024-10-25 00:00:00,NaN
Precio (€),59189.0,7.953348,0.69,1.39,3.83,9.78,64.59,10.775573
Variación,59189.0,-0.004951,-20.69,0.0,0.0,0.0,20.69,0.529826


In [22]:
df_total.describe(include="O").T

,count,unique,top,freq
supermercado,59189,6,eroski,23674
producto,59189,3,leche,30997
variante,59189,702,aceite-de-oliva-virgen-extra-picual-casa-junca...,207


COMENTARIOS EDA inicial:  



- La media de precio de los principales supermercados (en los productos de aceite de girasol, aceite de oliva y leche) en España es de 7,95€, mientras que la mediana es de 3,83€ indicando que en los principales supermercados tienen ciertos productos bastante más caros mientras que parece haber una mayor cantidad de productos "baratos" (pues para calcular la mediana se ordenann de menor a mayor los productos más baratos están al principio).

- En cuanto a la variación de precio, se observa que por lo general no hay variaciones y son bastante estables sin enbargo si que comprobamos un máximo y un mínimo de variación muy destacables(lo que podría indicar o bien promociones (como el descuento del IVA) o/y subidas de precio abusivas)

Otros datos de interés:  

- El supermercado que más aparece es Eroski también el producto Leche y la variante Aceite de Oliva Virgen Extra Casa Juncal.	


**CREACION DE BASE DE DATOS EN DBEAVER**

Primero diseño los Data Frames que serán mis futuras tablas en la la base de datos

In [23]:
df_supermercados= pd.DataFrame()
df_supermercados["super"]= df_total["supermercado"].unique()

df_productos= pd.DataFrame()
df_productos["producto"]= df_total["producto"].unique()

In [24]:
df_diccsupe=f.crear_df_id (df_supermercados, "super")


In [25]:
df_diccpro=f.crear_df_id(df_productos, "producto")

In [26]:
df_principal= pd.DataFrame()
df_principal["fecha"]= df_total["Día"]
df_principal["supermercado"]= df_total["supermercado"]
df_principal["producto"]= df_total["producto"]
df_principal["variante"]=df_total["variante"]
df_principal["precio"]= df_total["Precio (€)"]
df_principal["variacion"]=df_total["Variación"]
df_principal.head()

,fecha,supermercado,producto,variante,precio,variacion
0,2024-07-12,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l,21.4,0.0
1,2024-07-13,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l,21.4,0.0
2,2024-07-14,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l,21.4,0.0
3,2024-07-15,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l,21.4,0.0
4,2024-07-16,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l,21.4,0.0


In [28]:
f.añadir_id_mapeado(df_diccpro,"producto","id_producto","producto","id_producto", df_principal).sample(3)

,fecha,supermercado,variante,precio,variacion,id_producto
17447,2024-08-19,carrefour,leche-semidesnatada-pascual-sin-lactosa-brik-1...,1.53,0.0,3
9240,2024-08-13,eroski,aceite-de-oliva-virgen-guillen-botella-1-litro,8.30,0.0,2
208,2024-08-16,carrefour,aceite-de-girasol-carrefour-garrafa-5-l,10.55,0.0,1


In [29]:
f.añadir_id_mapeado(df_diccsupe,"super","id_super","supermercado","id_supermercado", df_principal).sample(3)

,fecha,variante,precio,variacion,id_producto,id_supermercado
6710,2024-08-22,aceite-de-oliva-virgen-extra-eco-oro-de-génabe...,11.42,0.0,2,3
3383,2024-09-15,aceite-de-oliva-intenso-coosur-botella-1-litro,11.49,0.0,2,3
14841,2024-07-24,leche-entera-puleva-sin-gluten-sin-lactosa-bot...,1.09,0.0,3,1


In [47]:
df_principal.index = pd.RangeIndex(start=1,stop = len(df_principal)+1,step=1)
df_principal["id_principal"]=df_principal.index


In [49]:
df_principal.head(2)

,fecha,variante,precio,variacion,id_producto,id_supermercado,id_principal
1,2024-07-12,aceite-de-girasol-capicua-garrafa-5-l,21.4,0.0,1,1,1
2,2024-07-13,aceite-de-girasol-capicua-garrafa-5-l,21.4,0.0,1,1,2


In [48]:
df_principal.sample(5)

,fecha,variante,precio,variacion,id_producto,id_supermercado,id_principal
28908,2024-09-14,aceite-de-oliva-virgen-extra-eco-coosur-botell...,13.76,0.0,2,3,28908
24150,2024-09-08,aceite-de-oliva-04-carbonell-lata-5-litros,50.43,0.0,2,3,24150
8515,2024-09-02,aceite-de-oliva-virgen-extra-marqués-de-griñón...,11.69,0.0,2,1,8515
58586,2024-09-16,aceites-de-ardales-aceite-de-oliva-virgen-extr...,7.65,0.0,2,5,58586
53174,2024-09-29,leche-entera-calcio-puleva-brik-1-l,5.16,0.0,3,4,53174


Ahora conecto con mi base de datos para empezar a crear allí las tablas con los datos

In [54]:
try:
    conexion= psycopg2.connect(
        database= "Precios_productos_basicos",
        user= "postgres",
        password= "admin",
        host= "localhost",
        port= "5432"
    )
except OperationalError as e:
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print("La contraseña es erronea")
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print("Error de conexion")
    else:
        print(f"Ocurrió el error {e}")

TABLA Supers

In [34]:
cursor= conexion.cursor()
query_creacion= """
CREATE TABLE IF NOT EXISTS mercados(
    super VARCHAR(100),
    id_super INT PRIMARY KEY   
);
"""
cursor.execute(query_creacion)   
conexion.commit()

In [35]:
lista_de_tuplas=[]
for fila in df_diccsupe.values:
    lista_de_tuplas.append(tuple(fila))


In [36]:
     
query_insercion= "INSERT INTO mercados (super, id_super) VALUES (%s, %s);"      #%s es como el format de los strings (la f)

cursor.executemany(query_insercion, lista_de_tuplas)                 #Executemany es para poder insertar varios valores
conexion.commit()

TABLA PRODUCTOS

In [43]:
cursor=conexion.cursor()
query_creacion= """
CREATE TABLE productos(
    producto VARCHAR(100),
    id_producto INT PRIMARY KEY
);
""" 
cursor.execute(query_creacion)
conexion.commit()

In [44]:
lista_de_tuplas= []
for fila in df_diccpro.values:
    lista_de_tuplas.append(tuple(fila))

In [45]:
query_insercion= "INSERT INTO productos(producto, id_producto)  VALUES (%s,%s);"
cursor.executemany(query_insercion, lista_de_tuplas)
conexion.commit()

TABLA PRINCIPAL

In [55]:
cursor=conexion.cursor()
query_creacion= """
CREATE TABLE principal(
    fecha DATE,
    variante VARCHAR(300),
    precio FLOAT,
    variacion FLOAT,
    id_producto INT,
    id_super INT,
    id_principal INT PRIMARY KEY,
    FOREIGN KEY (id_producto) REFERENCES productos(id_producto) ON DELETE RESTRICT ON UPDATE CASCADE,
    FOREIGN KEY (id_super) REFERENCES mercados(id_super) ON DELETE RESTRICT ON UPDATE CASCADE
);
""" 
cursor.execute(query_creacion)
conexion.commit()


In [56]:
lista_de_tuplas= []
for fila in df_principal.values:
    lista_de_tuplas.append(tuple(fila))

In [57]:
query_insercion= "INSERT INTO principal (fecha, variante, precio, variacion, id_producto, id_super, id_principal) VALUES (%s, %s, %s, %s, %s, %s,%s);"      #%s es como el format de los strings (la f)

cursor.executemany(query_insercion, lista_de_tuplas)                 #Executemany es para poder insertar varios valores
conexion.commit()